In [94]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import LabelEncoder

In [95]:
x_train = pd.read_csv('x_train.csv', parse_dates=['date_recorded'])
y_train = pd.read_csv('y_train.csv')
x_test = pd.read_csv('x_test.csv')
x_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [96]:
le = LabelEncoder()
labels = le.fit_transform(y_train['status_group'])
y_train = labels

In [97]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null datetime64[ns]
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null 

### Remove Nulls

In [98]:
x_train.scheme_management.unique()
x_train['scheme_management'] = x_train['scheme_management'].fillna('None')
x_train = pd.get_dummies(x_train, columns=['scheme_management'])

In [99]:
le_sn = LabelEncoder()
le_sn_labels = le_sn.fit_transform(x_train['scheme_name'].astype(str))
x_train['scheme_name'] = le_sn_labels

In [100]:
le_funding = LabelEncoder()
le_funding_labels = le_funding.fit_transform(x_train['funder'].astype(str))
x_train['funder'] = le_funding_labels

In [101]:
le_installer = LabelEncoder()
le_installer_labels = le_installer.fit_transform(x_train['installer'].astype(str))
x_train['installer'] = le_installer_labels

In [102]:
x_train['public_meeting'] = x_train['public_meeting'].fillna(False)

In [103]:
x_train['permit'] = x_train['permit'].fillna(False)

In [104]:
le_subvillage = LabelEncoder()
le_subvillage_labels = le_subvillage.fit_transform(x_train['subvillage'].astype(str))
x_train['subvillage'] = le_subvillage_labels

In [105]:
x_train = x_train.drop(['wpt_name', 'recorded_by'], axis=1)

In [106]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 49 columns):
id                                    59400 non-null int64
amount_tsh                            59400 non-null float64
date_recorded                         59400 non-null datetime64[ns]
funder                                59400 non-null int64
gps_height                            59400 non-null int64
installer                             59400 non-null int64
longitude                             59400 non-null float64
latitude                              59400 non-null float64
num_private                           59400 non-null int64
basin                                 59400 non-null object
subvillage                            59400 non-null int64
region                                59400 non-null object
region_code                           59400 non-null int64
district_code                         59400 non-null int64
lga                                   59400 non-

### Deal with Categoricals

In [107]:
le_lga = LabelEncoder()
le_lga_labels = le_lga.fit_transform(x_train['lga'].astype(str))
x_train['lga'] = le_lga_labels

In [108]:
le_ward = LabelEncoder()
le_ward_labels = le_ward.fit_transform(x_train['ward'].astype(str))
x_train['ward'] = le_ward_labels

In [109]:
x_train = pd.get_dummies(x_train, columns=['basin', 'region', 'extraction_type', 
                                           'extraction_type_group', 'extraction_type_class',
                                          'management', 'management_group', 'payment', 'payment_type',
                                          'water_quality', 'quality_group', 'quantity', 'quantity_group',
                                          'source', 'source_type', 'source_class', 'waterpoint_type', 
                                           'waterpoint_type_group'])

In [110]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Columns: 187 entries, id to waterpoint_type_group_other
dtypes: bool(2), datetime64[ns](1), float64(3), int64(13), uint8(168)
memory usage: 17.3 MB
